In [ ]:
!pip install ultralytics deep_sort_realtime opencv-python --quiet

In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
import os

In [ ]:
input_video_path = '/content/palace.mp4'

In [ ]:
model = YOLO('yolov8l.pt')  # YOLOv8 Nano for speed
tracker = DeepSort(max_age=35)

100%|██████████| 83.7M/83.7M [00:00<00:00, 217MB/s]


In [ ]:
cap = cv2.VideoCapture(input_video_path)

# Output video settings
output_path = 'output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

# Data Structures
seen_objects = {}   # object_id : last_seen_frame
new_objects = set()
missing_objects = set()
frame_count = 0

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    current_ids = set()

    # Step 1: Detect objects
    results = model.predict(source=frame, conf=0.5, verbose=False)[0]

    # Prepare detections for DeepSORT
    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))

    # Step 2: Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Step 3: Analyze tracks
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        current_ids.add(track_id)

        # New object detection
        if track_id not in seen_objects:
            print(f"[Frame {frame_count}] New object detected: ID {track_id}")
            new_objects.add(track_id)

        # Update last seen
        seen_objects[track_id] = frame_count

        # Draw bounding box
        l, t, w_, h_ = track.to_ltrb()
        cv2.rectangle(frame, (int(l), int(t)), (int(w_), int(h_)), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (int(l), int(t) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Step 4: Check for missing objects
    for obj_id in list(seen_objects.keys()):
        if obj_id not in current_ids:
            if frame_count - seen_objects[obj_id] > 30:
                if obj_id not in missing_objects:
                    print(f"[Frame {frame_count}] Missing object detected: ID {obj_id}")
                    missing_objects.add(obj_id)

    # Step 5: Write output frame
    out.write(frame)

cap.release()
out.release()

print("✅ Video processing complete. Saving file...")

[Frame 3] New object detected: ID 1
[Frame 3] New object detected: ID 2
[Frame 3] New object detected: ID 3
[Frame 3] New object detected: ID 4
[Frame 3] New object detected: ID 5
[Frame 3] New object detected: ID 6
[Frame 3] New object detected: ID 7
[Frame 3] New object detected: ID 8
[Frame 3] New object detected: ID 9
[Frame 3] New object detected: ID 10
[Frame 3] New object detected: ID 11
[Frame 3] New object detected: ID 12
[Frame 3] New object detected: ID 14
[Frame 4] New object detected: ID 16
[Frame 7] New object detected: ID 17
[Frame 10] New object detected: ID 18
[Frame 11] New object detected: ID 19
[Frame 14] New object detected: ID 20
[Frame 31] New object detected: ID 31
[Frame 33] New object detected: ID 33
[Frame 38] New object detected: ID 34
[Frame 51] New object detected: ID 36
[Frame 57] New object detected: ID 37
[Frame 67] New object detected: ID 39
[Frame 71] New object detected: ID 40
[Frame 76] New object detected: ID 41
[Frame 81] Missing object detected: 